In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mtcnn opencv-python-headless torch torchvision imageio[pyav] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2

In [3]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import imageio.v3 as iio
from tqdm import tqdm
from pathlib import Path
from torchvision import models, transforms
from mtcnn import MTCNN

In [4]:
base_path = Path("/content/drive/MyDrive/DeepFake Detection")
video_dir = base_path / "merged_dataset"
metadata_path = base_path / "metadata.csv"
output_dir = base_path / "processed_features/strategy_2"
output_dir.mkdir(parents=True, exist_ok=True)

In [5]:
metadata_df = pd.read_csv(metadata_path)
label_map = {"FAKE": "Fake", "REAL": "Real"}
metadata_df["full_path"] = metadata_df.apply(lambda row: video_dir / label_map[row["label"]] / row["filename"], axis=1)

In [6]:
face_detector = MTCNN()
model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.DEFAULT)
model.fc = torch.nn.Identity()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth
100%|██████████| 95.8M/95.8M [00:01<00:00, 91.8MB/s]


In [7]:
def sample_frames(frames):
    total = len(frames)
    if total == 0: return []
    if total < 10: return frames
    idxs = np.arange(0, total, total // 10)[:10]
    return [frames[i] for i in idxs]

In [8]:
def extract_frames(video_path):
    try:
        frames = list(iio.imread(video_path, plugin="pyav"))
        return sample_frames(frames)
    except Exception as e:
        print(f"🚫 Failed reading {video_path.name}: {e}")
        return []

In [9]:
@torch.no_grad()
def get_features(faces):
    if len(faces) == 0: return None
    tensors = torch.stack([transform(face) for face in faces]).to(device)
    features = model(tensors)
    return features.mean(dim=0).cpu().numpy()

In [10]:
for _, row in tqdm(metadata_df.iterrows(), total=len(metadata_df), desc="Strategy 2"):
    filename = row["filename"]
    video_path = row["full_path"]
    out_path = output_dir / filename.replace(".mp4", ".npy")

    if out_path.exists():
        continue

    frames = extract_frames(video_path)
    if not frames: continue

    faces = []
    for frame in frames:
        res = face_detector.detect_faces(frame)
        if res:
            x, y, w, h = res[0]["box"]
            face = frame[y:y+h, x:x+w]
            if face.size > 0: faces.append(face)

    if not faces: continue
    features = get_features(faces)
    if features is None: continue

    np.save(out_path, features)

print("\n✅ Strategy 2 complete. Features saved as .npy")

Strategy 2: 100%|██████████| 2538/2538 [55:30<00:00,  1.31s/it]


✅ Strategy 2 complete. Features saved as .npy
